stratifed Kfold, lr 스케줄링, 

batchnorm 대신 groupnorm

https://blog.airlab.re.kr/2019/08/Group-Normalization,

https://github.com/ppwwyyxx/GroupNorm-reproduce/blob/master/ImageNet-ResNet-PyTorch/resnet_gn.py

https://nbviewer.jupyter.org/github/amaarora/amaarora.github.io/blob/master/nbs/Group%20Normalization.ipynb
 

 1. train_test_split+baseline cnn
 2. kfold + baseline cnn
 3. kfold+사전학습모델

In [1]:
!Is-Iha kaggle.json
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.10

/bin/bash: Is-Iha: command not found
Uninstalling kaggle-1.5.10:
  Successfully uninstalled kaggle-1.5.10
     |████████████████████████████████| 1.5MB 8.2MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 59 kB 3.9 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-py3-none-any.whl size=73268 sha256=7f94125cf64b28d403fa5349a4e5c394219142b0c1d831ecc026fae4483af94e
  Stored in directory: /root/.cache/pip/wheels/ea/c5/fe/7e7fb5b3d1f150fac96188949b3d83d375a4c9df16ba557e52
Successfully built kaggle


In [2]:
#json파일 확인
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle -v

Kaggle API 1.5.10


In [3]:
!kaggle competitions download -c sejong-ai-challenge-p1

 72% 17.0M/23.8M [00:00<00:00, 42.2MB/s]
100% 23.8M/23.8M [00:00<00:00, 86.3MB/s]


In [4]:
!unzip sejong-ai-challenge-p1

Archive:  sejong-ai-challenge-p1.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [5]:
import numpy as np
import pandas as pd
import random
import torch
import time
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

In [6]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

missing, NAN 처리

In [7]:
train['label'].unique()

array(['missing', '9', '6', 'NAN', '3', '4', '5', '8', '7', '1', '0'],
      dtype=object)

In [8]:
train[train['label']=='missing']=2
train[train['label']=='NAN']=np.nan

In [9]:
train['label'].isna().sum()

87

In [10]:
train=train.dropna()

In [11]:
train['label'].isna().sum()

0

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [13]:
train

,id,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,label
0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,105.0,92.0,101.0,107.0,100.0,132.0,0.0,0.0,2.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150.0,...,220.0,214.0,74.0,0.0,255.0,222.0,128.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,12.0,0.0,0.0,40.0,134.0,162.0,191.0,214.0,163.0,146.0,165.0,79.0,0.0,0.0,0.0,30.0,43.0,0.0,0.0,0.0,0.0,0.0,6
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,0.0,21.0,68.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,187.0,189.0,...,237.0,229.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68.0,116.0,112.0,136.0,147.0,144.0,121.0,102.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
5,5.0,0.0,0.0,0.0,5.0,4.0,5.0,5.0,3.0,5.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,7.0,4.0,4.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,4.0,5.0,11.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,4.0,9.0,7.0,8.0,7.0,5.0,4.0,8.0,12.0,5.0,7.0,8.0,7.0,4.0,3.0,7.0,5.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33835,59990.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,19.0,221.0,214.0,203.0,202.0,200.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,0.0,0.0,0.0,0.0,...,242.0,229.0,223.0,240.0,104.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,19.0,184.0,181.0,170.0,173.0,170.0,169.0,171.0,170.0,161.0,158.0,154.0,161.0,165.0,83.0,0.0,5.0,0.0,0.0,0.0,0.0,0
33836,59991.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
33837,59992.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
33838,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0

In [14]:
x_train=train.loc[:,'pixel1':'pixel784']
y_train=train['label']


x_test=test.loc[:,'pixel1':'pixel784']

In [15]:
x_train=np.array(x_train)
y_train=np.array(y_train,dtype=int)

x_test=np.array(x_test)

In [16]:
x_train=torch.FloatTensor(x_train)
y_train=torch.LongTensor(y_train)

x_test=torch.FloatTensor(x_test)


In [17]:
x_train = x_train.reshape(-1,1, 28, 28)

x_test=x_test.reshape(-1,1,28,28)

In [18]:
x_train.shape

torch.Size([33753, 1, 28, 28])

In [ ]:
x_train.size()[1:]

torch.Size([1, 28, 28])

#1. train, valid 그냥 split

In [26]:
# 학습 파라미터 설정
learning_rate = 1e-3
epochs = 20
batch_size = 50


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val= train_test_split(x_train,y_train,test_size=0.3,random_state=1)

In [ ]:
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)

val_dataset=torch.utils.data.TensorDataset(x_val,y_val)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

val_loader=torch.utils.data.DataLoader(dataset=val_dataset,
                                       batch_size=batch_size,
                                       shuffle=True,
                                       drop_last=True
                                       )


##base CNN model with 1. batchNorm ,2. groupNorm

In [36]:

class baseline_CNN(torch.nn.Module):
  def __init__(self):
    super(baseline_CNN,self).__init__() 

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
        torch.nn.BatchNorm2d(64),
        #torch.nn.GroupNorm(32,128),# group, channel
        torch.nn.ReLU(),
        torch.nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1),
        torch.nn.BatchNorm2d(64),
        #torch.nn.GroupNorm(32,128),# group, channel
        torch.nn.ReLU(),
        #torch.nn.Dropout2d(0.3),
        torch.nn.MaxPool2d(kernel_size=2,stride=2,padding=0) #14*14
    )
#layernormalization 
   
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        torch.nn.BatchNorm2d(128),
        #torch.nn.GroupNorm(32,128),# group, channel
        torch.nn.ReLU(),
        torch.nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
        torch.nn.BatchNorm2d(128),
        #torch.nn.GroupNorm(32,128),# group, channel
        torch.nn.ReLU(),
        #torch.nn.Dropout2d(0.3),
        torch.nn.MaxPool2d(kernel_size=2,stride=2,padding=0) #7*7
    )
    

 
    self.avgpool=torch.nn.AdaptiveAvgPool2d(output_size=(7,7))

    self.fc = torch.nn.Sequential(torch.nn.Linear(7*7*128,128,bias=True),
                                  torch.nn.ReLU(),
                                  torch.nn.Dropout2d(0.3),
                                  torch.nn.Linear(128,128,bias=True),
                                  torch.nn.ReLU(),
                                  torch.nn.Dropout2d(0.3),
                                  torch.nn.Linear(128,10,bias=True)
                                  )
  

  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out=self.avgpool(out)
    out = out.view(out.size(0),-1)     
    out = self.fc(out)
    return out

In [ ]:
model = baseline_CNN().to(device)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.


In [ ]:
loss_fn = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler =torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=250)

In [ ]:
# train loss와 val loss 지정.
total_loss = 0.0
total_val_loss = 0.0

# 모델 이름을 위해서 변수 만듦.
version = time.localtime()[3:5]
curr_lr = learning_rate

n_val_loss = 10000000. # 가장 낮은 validation loss를 저장하기 위해서 변수 설정.

In [ ]:
total_step = len(train_loader) * epochs
print(f"Total step is....{total_step}") # 모델이 학습하는 전체 step 계산.

Total step is....5660


In [ ]:
for epoch in range(epochs):
    total_loss=0
    total_val_loss=0

    for i,(X_train, Y_train) in enumerate(tqdm(train_loader, desc='*********Train mode*******')):
        if torch.cuda.is_available():
          
            X_train = X_train.float().to(device)
            Y_train = Y_train.long().to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X_train)
        # Error 계산
        loss = loss_fn(hypothesis, Y_train)
        # Backparopagation
        loss.backward()
        # 가중치 갱신
        optimizer.step()
        #lr 스케줄러 갱신
        scheduler.step()

        total_loss += loss.item()
        # 평균 Error 계산
        torch.cuda.empty_cache()
        
    train_loss = total_loss / len(train_loader)
    
    print ("Epoch [{}/{}], Train Loss: {:.4f}".format(epoch+1, epochs, train_loss))



    model.eval()

    with torch.no_grad():
      for i,(X_val, Y_val) in enumerate(tqdm(val_loader, desc='*********Evaluation mode*******')):
        X_val=X_val.float().to(device)
        Y_val=Y_val.long().to(device)

        hypothesis = model(X_val)
        loss_val = loss_fn(hypothesis,Y_val)

       

        total_val_loss += loss_val.item()
       


    val_loss = total_val_loss / len(val_loader)
    print ("Epoch [{}/{}], Eval Loss: {:.4f}".format(epoch+1, epochs, val_loss))


    if val_loss < n_val_loss:
      n_val_loss = val_loss
      torch.save(model.state_dict(), f'test_{version}_{learning_rate}_{epochs}.pth')
      print("Best Model saved......")
      



Epoch [1/20], Train Loss: 0.6165



Epoch [1/20], Eval Loss: 0.4240
Best Model saved......



Epoch [2/20], Train Loss: 0.3103



Epoch [2/20], Eval Loss: 0.3027
Best Model saved......



Epoch [3/20], Train Loss: 0.2672


...


Epoch [18/20], Eval Loss: 0.2793



Epoch [19/20], Train Loss: 0.0599



Epoch [19/20], Eval Loss: 0.2472



Epoch [20/20], Train Loss: 0.0472



Epoch [20/20], Eval Loss: 0.2613


In [ ]:
x_test=x_test.to(device)

test_model = baseline_CNN()

weights = torch.load(f'test_{version}_{learning_rate}_{epochs}.pth', map_location='cuda:0')
#모델


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.


In [ ]:
test_model.load_state_dict(weights)
test_model = test_model.to(device)
test_model.eval()

baseline_CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): GroupNorm(32, 128, eps=1e-05, affine=True)
    (2): ReLU()
    (3): Dropout2d(p=0.3, inplace=False)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): GroupNorm(32, 256, eps=1e-05, affine=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=12544, out_features=10, bias=True)
)

In [ ]:
with torch.no_grad():
   outputs = test_model(x_test)
pred_test = outputs

In [ ]:
pred_test

tensor([[  3.2795,   0.2898, -30.5480,  ...,  -0.4665,   8.6076,  -5.9985],
        [  8.4842,   3.4031, -26.7545,  ...,  -4.3272,  -5.0022,  -8.6916],
        [  6.9383,   0.7098, -27.0416,  ...,  -3.6872,  -6.8897, -10.7793],
        ...,
        [  3.4938,  -2.3058, -26.2242,  ...,   2.4306,  -5.3481,  -4.8173],
        [  4.8001,   1.2742, -27.9954,  ...,  -6.2156,  -3.5192,  -5.2744],
        [  3.0364,  13.9015, -23.1632,  ...,  -3.9279,  -1.2535,  -6.1607]],
       device='cuda:0')

In [ ]:
prediction=pred_test.argmax(1)

In [ ]:
prediction=prediction.cpu()

In [ ]:
submit=pd.read_csv("sample_submission.csv")

for i in range(len(submit)):
    submit['label']=prediction

submit

,id,label
0,11,8
1,15,0
2,17,3
3,21,7
4,22,4
...,...,...
8455,59970,7
8456,59971,5
8457,59982,5
8458,59986,6


In [ ]:
submit.to_csv("submission.csv",index=False)

In [ ]:
!kaggle competitions submit -c sejong-ai-challenge-p1 -f submission.csv -m "Message"

100% 64.5k/64.5k [00:03<00:00, 18.8kB/s]
Successfully submitted to Sejong AI Challenge 문제1

##base cnn: 0.78~79

#2. KFold + baselinecnn

In [27]:
x_train=train.loc[:,'pixel1':'pixel784']
y_train=train['label']
y_train=y_train.astype(int)

x_test=test.loc[:,'pixel1':'pixel784']
#x_train=np.array(x_train)
#y_train=np.array(y_train,dtype=int)

#x_test=np.array(x_test)

#x_train=torch.FloatTensor(x_train)
#y_train=torch.LongTensor(y_train)

#x_test=torch.FloatTensor(x_test)


#x_train = x_train.reshape(-1,1, 28, 28)

#x_test=x_test.reshape(-1,1,28,28)

In [28]:
from sklearn.model_selection import StratifiedKFold

In [30]:

random_state = 777
n_folds = 4
k_fold = StratifiedKFold(n_splits=n_folds, random_state=random_state, shuffle=True)

In [37]:
model=baseline_CNN()

In [38]:
epochs=7
learning_rate=1e-3

In [39]:
def train_one_fold(i_fold, model, loss_func, optimizer, train_loader, valid_loader):
    
    train_fold_results = []

    for epoch in range(epochs):

        print('  Epoch {}/{}'.format(epoch + 1, epochs))
        print('  ' + ('-' * 20))

        model.train()
        tr_loss = 0
        train_kappa = []

        train_accuracy=[]
    
        optimizer.zero_grad()
        for ii, (data, target) in enumerate(train_loader):

            images = data.to(device, dtype = torch.float)  
#             target = target.unsqueeze(1)
            labels = target.to(device, dtype = torch.long)  
            
            #labels = labels.view(-1,1)
            optimizer.zero_grad()
            
            
            with torch.set_grad_enabled(True):
                output = model(images)   
                
                loss = loss_func(output, labels)
                loss.backward()

                optimizer.step()
                scheduler.step()
            
            tr_loss += loss.item()
            
            y_actual = labels.data.cpu().numpy()
            y_pred = output[:,-1].detach().cpu().numpy()
            y_pred1=output.argmax(1).cpu()
            #kappa = cohen_kappa_score(y_actual, y_pred.round(), weights='quadratic')
            accuracy=accuracy_score(y_actual,y_pred1)


            #train_kappa.append(kappa)
            train_accuracy.append(accuracy)
        
        #train_kappa_epoch = np.mean(train_kappa)
        train_accuracy_epoch=np.mean(train_accuracy)
            

        # Validate
        model.eval()
        
        val_loss = 0
        val_preds = None
        val_labels = None
        val_kappa = []
        
        val_accuracy=[]

        for ii, (data, target) in enumerate(valid_loader):


            images = data.to(device, dtype = torch.float)
            labels = target.to(device, dtype = torch.long)           
            #labels = labels.view(-1,1)

            with torch.no_grad():
                outputs = model(images)
                
                loss = loss_func(outputs, labels)
                val_loss += loss.item()

            y_actual = labels.data.cpu().numpy()
            y_pred = outputs[:,-1].detach().cpu().numpy()
            y_pred1=outputs.argmax(1).cpu()
            #kappa = cohen_kappa_score(y_actual, y_pred.round(), weights='quadratic')
            #val_kappa.append(kappa)
            accuaracy=accuracy_score(y_actual,y_pred1)
            val_accuracy.append(accuracy)


        #val_kappa_epoch = np.mean(val_kappa)
        val_accuracy_epoch=np.mean(val_accuracy)
        train_loss = tr_loss/len(train_loader)
        valid_loss = val_loss/len(valid_loader)
        
        print('Fold: {}, Epoch: {}, Train Loss: {:.6f}, Valid Loss: {:.6f}, Train accuracy: {:.4f}, Valid accuracy: {:.4f}'.format(i_fold+1, epoch+1,train_loss, valid_loss, train_accuracy_epoch, val_accuracy_epoch))

In [40]:
for i_fold, (train_idx, valid_idx) in enumerate(k_fold.split(x_train, y_train)):
    print("Fold {}/{}".format(i_fold + 1,n_folds))

    print(valid_idx)
    print(train_idx)


    fold_x_valid = x_train.iloc[valid_idx]
    fold_x_valid.reset_index(drop=True, inplace=True)
    fold_y_valid=y_train.iloc[valid_idx]

    fold_x_train = x_train.iloc[train_idx]
    fold_x_train.reset_index(drop=True, inplace=True)    
    fold_y_train=y_train.iloc[train_idx]
    
    fold_x_train=np.array(fold_x_train)
    fold_y_train=np.array(fold_y_train,dtype=int)

    fold_x_valid=np.array(fold_x_valid)
    fold_y_valid=np.array(fold_y_valid,dtype=int)

    fold_x_train=torch.FloatTensor(fold_x_train)
    fold_y_train=torch.LongTensor(fold_y_train)

    fold_x_valid=torch.FloatTensor(fold_x_valid)
    fold_y_valid=torch.LongTensor(fold_y_valid)


    fold_x_train = fold_x_train.reshape(-1,1, 28, 28)

    fold_x_valid=fold_x_valid.reshape(-1,1,28,28)

    train_dataset = torch.utils.data.TensorDataset(fold_x_train, fold_y_train)
    val_dataset=torch.utils.data.TensorDataset(fold_x_valid,fold_y_valid)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

    val_loader=torch.utils.data.DataLoader(dataset=val_dataset,
                                       batch_size=batch_size,
                                       shuffle=True,
                                       drop_last=True
                                       )
    
    model = model.to(device)


    loss_func = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler =torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=250)

    train_one_fold(i_fold, model, loss_func, optimizer, train_loader, val_loader)


torch.save(model.state_dict(), 'aptos_3_fold_weights.pth')
    

Fold 1/4
[    1     2    10 ... 33748 33750 33752]
[    0     3     4 ... 33747 33749 33751]
  Epoch 1/7
  --------------------
Fold: 1, Epoch: 1, Train Loss: 0.361871, Valid Loss: 0.320170, Train accuracy: 0.8602, Valid accuracy: 0.8200
  Epoch 2/7
  --------------------
Fold: 1, Epoch: 2, Train Loss: 0.244257, Valid Loss: 0.201585, Train accuracy: 0.9036, Valid accuracy: 0.9000
  Epoch 3/7
  --------------------
Fold: 1, Epoch: 3, Train Loss: 0.203827, Valid Loss: 0.226472, Train accuracy: 0.9209, Valid accuracy: 0.8800
  Epoch 4/7
  --------------------
Fold: 1, Epoch: 4, Train Loss: 0.186995, Valid Loss: 0.192101, Train accuracy: 0.9266, Valid accuracy: 0.9400
  Epoch 5/7
  --------------------
Fold: 1, Epoch: 5, Train Loss: 0.175131, Valid Loss: 0.179713, Train accuracy: 0.9305, Valid accuracy: 0.9400
  Epoch 6/7
  --------------------
Fold: 1, Epoch: 6, Train Loss: 0.162561, Valid Loss: 0.170661, Train accuracy: 0.9372, Valid accuracy: 0.9200
  Epoch 7/7
  --------------------
Fo

In [41]:
test=pd.read_csv("test.csv")

test['label']=-1
x_test=test.loc[:,'pixel1':'pixel784']
x_test_label=test['label']

In [42]:
x_test=np.array(x_test)
x_test=torch.FloatTensor(x_test)
x_test_label=torch.LongTensor(x_test_label)

x_test=x_test.reshape(-1,1,28,28)

test_dataset = torch.utils.data.TensorDataset(x_test,x_test_label)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

실행안됨 ㅠㅠ

In [ ]:
def predict(testloader):
    '''Function used to make predictions on the test set'''
    model.eval()
    preds = []
    for batch_i, (data, target) in enumerate(testloader):
        data, target = data.cuda(), target.cuda()
        output = model(data)
        pr = output.detach().cpu().numpy()
        for i in pr:
            preds.append(i.item()) 
            print(i)
            print(i.item())

            
    return preds
   

In [ ]:
preds1 = np.array(predict(testloader=test_loader))

ValueError: ignored

기존의 test code

In [47]:
x_test=x_test.to(device)

test_model = baseline_CNN()

weights = torch.load(f'aptos_3_fold_weights.pth', map_location='cuda:0')


test_model.load_state_dict(weights)
test_model = test_model.to(device)
test_model.eval()


with torch.no_grad():
   outputs = test_model(x_test)
pred_test = outputs



In [48]:
pred_test

tensor([[ -5.3158, -11.4864, -24.6991,  ...,  -7.7596,   6.5196, -13.7512],
        [  6.2912, -14.3342, -38.3514,  ..., -23.4064, -28.4445, -32.0468],
        [ -2.4169,  -9.8538, -26.4975,  ..., -11.8875, -20.0570, -19.7966],
        ...,
        [-18.1798, -17.1572, -35.3632,  ..., -12.3196, -21.6955, -17.8085],
        [ -2.9860,  -7.6020, -17.9474,  ..., -11.5156,  -8.1861, -11.2789],
        [-11.7387,  12.2524, -37.7722,  ..., -28.5703, -46.0268, -29.4307]],
       device='cuda:0')

In [49]:
prediction=pred_test.argmax(1).cpu()

In [50]:
prediction

tensor([8, 0, 3,  ..., 5, 6, 1])

In [51]:
submit=pd.read_csv("sample_submission.csv")

for i in range(len(submit)):
    submit['label']=prediction

submit

,id,label
0,11,8
1,15,0
2,17,3
3,21,7
4,22,4
...,...,...
8455,59970,7
8456,59971,5
8457,59982,5
8458,59986,6


In [52]:
submit['label'].unique()

array([8, 0, 3, 7, 4, 1, 5, 6, 9])

In [53]:
submit.to_csv("submission.csv",index=False)

In [54]:
!kaggle competitions submit -c sejong-ai-challenge-p1 -f submission.csv -m "Message"

100% 64.5k/64.5k [00:02<00:00, 24.7kB/s]
Successfully submitted to Sejong AI Challenge 문제1

성능 0.8

##사전학습 -> 사진크기너무 작음(resize로키우던가해야함..)

In [19]:
entrypoints = torch.hub.list('pytorch/vision', force_reload=True)

Downloading: "https://github.com/pytorch/vision/archive/master.zip" to /root/.cache/torch/hub/master.zip


In [20]:
entrypoints

['alexnet',
 'deeplabv3_mobilenet_v3_large',
 'deeplabv3_resnet101',
 'deeplabv3_resnet50',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'fcn_resnet101',
 'fcn_resnet50',
 'googlenet',
 'inception_v3',
 'lraspp_mobilenet_v3_large',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet_v2',
 'mobilenet_v3_large',
 'mobilenet_v3_small',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'squeezenet1_0',
 'squeezenet1_1',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn',
 'wide_resnet101_2',
 'wide_resnet50_2']

In [23]:
repo = 'pytorch/vision'
pretrained_model = torch.hub.load(repo, 'vgg19_bn', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_master
Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth


In [24]:
print(pretrained_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [ ]:

#pretrained_model.features.conv0=torch.nn.Conv2d(1,64,kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#pretrained_model.conv1=torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

#pretrained_model.fc.out_features=9


pretrained_model.classifier[6].out_features = 9

for param in pretrained_model.features.parameters():
    param.requires_grad = False

In [ ]:
print(pretrained_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [ ]:
final_model=torch.nn.Sequential(torch.nn.Conv2d(1,3,kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
                                pretrained_model)

In [ ]:
#densenet201
# freeze the model weights
# for param in model.parameters():
#     param.requires_grad = False
    
# Resnet uses the name fc for its last layer while Densenet uses the name classifier for its last layer.
# model.avg_pool = nn.AdaptiveAvgPool2d(output_size=(1,1))
num_ftrs = densenet201.classifier.in_features
densenet201.classifier = torch.nn.Linear(num_ftrs, 9).float()
 #for BCEWithLogitsLoss() number of output neuron should be 1, for CrossEntropyLoss() number of output neuron should be equal to the number of classes

 # freeze convolution weights
for param in densenet201.features.parameters():
    param.requires_grad = False

In [ ]:


for i_fold, (train_idx, valid_idx) in enumerate(k_fold.split(x_train, y_train)):
    print("Fold {}/{}".format(i_fold + 1,n_folds))

    print(valid_idx)
    print(train_idx)


    fold_x_valid = x_train.iloc[valid_idx]
    fold_x_valid.reset_index(drop=True, inplace=True)
    fold_y_valid=y_train.iloc[valid_idx]

    fold_x_train = x_train.iloc[train_idx]
    fold_x_train.reset_index(drop=True, inplace=True)    
    fold_y_train=y_train.iloc[train_idx]
    
    fold_x_train=np.array(fold_x_train)
    fold_y_train=np.array(fold_y_train,dtype=int)

    fold_x_valid=np.array(fold_x_valid)
    fold_y_valid=np.array(fold_y_valid,dtype=int)

    fold_x_train=torch.FloatTensor(fold_x_train)
    fold_y_train=torch.LongTensor(fold_y_train)

    fold_x_valid=torch.FloatTensor(fold_x_valid)
    fold_y_valid=torch.LongTensor(fold_y_valid)


    fold_x_train = fold_x_train.reshape(-1,1, 28, 28)

    fold_x_valid=fold_x_valid.reshape(-1,1,28,28)

    train_dataset = torch.utils.data.TensorDataset(fold_x_train, fold_y_train)
    val_dataset=torch.utils.data.TensorDataset(fold_x_valid,fold_y_valid)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

    val_loader=torch.utils.data.DataLoader(dataset=val_dataset,
                                       batch_size=batch_size,
                                       shuffle=True,
                                       drop_last=True
                                       )
    
    model = final_model.to(device)


    loss_func = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler =torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=250)

    train_one_fold(i_fold, model, loss_func, optimizer, train_loader, val_loader)


torch.save(model.state_dict(), 'pretrained_model_densenet201_weights.pth')
    

Fold 1/5
[    1     4     9 ... 28944 28948 28952]
[    0     2     3 ... 28949 28950 28951]
  Epoch 1/7
  --------------------


RuntimeError: ignored

In [ ]:

fold_x_valid=fold_x_valid.reshape(-1,3,28,28)
fold_x_valid.shape

RuntimeError: ignored